In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import ast
import pickle, bz2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

# -------------------------------
# Load Dataset
# -------------------------------
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

movies = movies.merge(credits, on='title')
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.dropna(inplace=True)

# -------------------------------
# Helper functions
# -------------------------------
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

def fetch_director(obj):
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            return [i['name']]
    return []

def collapse(L):
    return [i.replace(" ", "") for i in L]

# -------------------------------
# Clean Data
# -------------------------------
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert3)
movies['crew'] = movies['crew'].apply(fetch_director)

movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

new = movies[['movie_id', 'title', 'tags']]
new['tags'] = new['tags'].apply(lambda x: " ".join(x).lower())

# -------------------------------
# Text Preprocessing (Stemming)
# -------------------------------
ps = PorterStemmer()

def stem(text):
    return " ".join([ps.stem(i) for i in text.split()])

new['tags'] = new['tags'].apply(stem)

# -------------------------------
# Vectorization + Similarity
# -------------------------------
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

similarity = cosine_similarity(vector)

# -------------------------------
# Save compressed files
# -------------------------------
with bz2.BZ2File("movies.pbz2", "w") as f:
    pickle.dump(new, f)

with bz2.BZ2File("similarity.pbz2", "w") as f:
    pickle.dump(similarity, f)

print("✅ Training complete. Files saved: movies.pbz2 & similarity.pbz2")


C:\Users\Sonu Gupta\AppData\Local\Temp\ipykernel_28884\475757731.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['tags'] = new['tags'].apply(lambda x: " ".join(x).lower())
C:\Users\Sonu Gupta\AppData\Local\Temp\ipykernel_28884\475757731.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['tags'] = new['tags'].apply(stem)


✅ Training complete. Files saved: movies.pbz2 & similarity.pbz2
